In [144]:
import pandas as pd
from sqlalchemy import create_engine
from config import dbConnectionString
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier

In [145]:
engine = create_engine(dbConnectionString)

#import dataframe from SQL database
df = pd.read_sql_query('select * from "cardioData"',con=engine)
df = df.drop(columns = ['index', 'id'])
df

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active
0,50.4,Female,168.0,62.0,False,23,110,80,normal,normal,False,False,True
1,55.4,Male,156.0,85.0,True,46,140,90,well above normal,normal,False,False,True
2,51.7,Male,165.0,64.0,True,25,130,70,well above normal,normal,False,False,False
3,47.9,Male,156.0,56.0,False,20,100,60,normal,normal,False,False,False
4,60.0,Male,151.0,67.0,False,30,120,80,above normal,above normal,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,Female,168.0,76.0,False,34,120,80,normal,normal,True,False,True
63912,61.9,Male,158.0,126.0,True,100,140,90,above normal,above normal,False,False,True
63913,52.2,Female,183.0,105.0,True,60,180,90,well above normal,normal,False,True,False
63914,61.5,Male,163.0,72.0,True,32,135,80,normal,well above normal,False,False,False


In [146]:
le = LabelEncoder()
df2 = df.copy()
#df2['age'] = df2['age'] x 365
df2

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active
0,50.4,Female,168.0,62.0,False,23,110,80,normal,normal,False,False,True
1,55.4,Male,156.0,85.0,True,46,140,90,well above normal,normal,False,False,True
2,51.7,Male,165.0,64.0,True,25,130,70,well above normal,normal,False,False,False
3,47.9,Male,156.0,56.0,False,20,100,60,normal,normal,False,False,False
4,60.0,Male,151.0,67.0,False,30,120,80,above normal,above normal,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,Female,168.0,76.0,False,34,120,80,normal,normal,True,False,True
63912,61.9,Male,158.0,126.0,True,100,140,90,above normal,above normal,False,False,True
63913,52.2,Female,183.0,105.0,True,60,180,90,well above normal,normal,False,True,False
63914,61.5,Male,163.0,72.0,True,32,135,80,normal,well above normal,False,False,False


In [147]:
df2['age'] = df2['age'].astype(float)

In [148]:
df2['BMI'] = df2['BMI'].astype(float)

In [149]:
df2['BMI_class'] = ''

for i in range(len(df2['age'])):
    if df2['BMI'][i] < 18.5:
        df2['BMI_class'][i] = 'underweight'
    elif df2['BMI'][i] < 24.9 and df2['BMI'][i] > 18.5:
        df2['BMI_class'][i] = 'normal'
    elif df2['BMI'][i] < 29.9 and df2['BMI'][i] > 24.9:
        df2['BMI_class'][i] = 'overweight'
    elif df2['BMI'][i] < 34.9 and df2['BMI'][i] > 29.9:
        df2['BMI_class'][i] = 'obese'
    elif df2['BMI'][i] < 39.9 and df2['BMI'][i] > 34.9:
        df2['BMI_class'][i] = 'obese2'
    else:
        df2['BMI_class'][i] = 'obese3'

df2['ap_hi_class'] = ''

for i in range(len(df2['age'])):
    if df2['ap_hi'][i] < 120:
        df2['ap_hi_class'][i] = 'normal'
    elif df2['ap_hi'][i] < 129 and df2['ap_hi'][i] >= 120:
        df2['ap_hi_class'][i] = 'elevated'
    elif df2['ap_hi'][i] < 139 and df2['ap_hi'][i] >= 130:
        df2['ap_hi_class'][i] = 'highbloodpressure'
    elif df2['ap_hi'][i] < 180 and df2['ap_hi'][i] >= 140:
        df2['ap_hi_class'][i] = 'highbloodpressure2'
    else:
        df2['ap_hi_class'][i] = 'hypertensive'

df2['ap_lo_class'] = ''

for i in range(len(df2['age'])):
    if df2['ap_lo'][i] < 80:
        df2['ap_lo_class'][i] = 'elevated'
    elif df2['ap_lo'][i] < 80 and df2['ap_lo'][i] >= 89:
        df2['ap_lo_class'][i] = 'highbloodpressure'
    elif df2['ap_lo'][i] < 120 and df2['ap_lo'][i] >= 90:
        df2['ap_lo_class'][i] = 'highbloodpressure2'
    else:
        df2['ap_lo_class'][i] = 'hypertensive'

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '

In [150]:
#female = 0, male = 1
df2['gender'] = le.fit_transform(df2['gender'])
df2

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active,BMI_class,ap_hi_class,ap_lo_class
0,50.4,0,168.0,62.0,False,23.0,110,80,normal,normal,False,False,True,normal,normal,hypertensive
1,55.4,1,156.0,85.0,True,46.0,140,90,well above normal,normal,False,False,True,obese3,highbloodpressure2,highbloodpressure2
2,51.7,1,165.0,64.0,True,25.0,130,70,well above normal,normal,False,False,False,overweight,highbloodpressure,elevated
3,47.9,1,156.0,56.0,False,20.0,100,60,normal,normal,False,False,False,normal,normal,elevated
4,60.0,1,151.0,67.0,False,30.0,120,80,above normal,above normal,False,False,False,obese,elevated,hypertensive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,0,168.0,76.0,False,34.0,120,80,normal,normal,True,False,True,obese,elevated,hypertensive
63912,61.9,1,158.0,126.0,True,100.0,140,90,above normal,above normal,False,False,True,obese3,highbloodpressure2,highbloodpressure2
63913,52.2,0,183.0,105.0,True,60.0,180,90,well above normal,normal,False,True,False,obese3,hypertensive,highbloodpressure2
63914,61.5,1,163.0,72.0,True,32.0,135,80,normal,well above normal,False,False,False,obese,highbloodpressure,hypertensive


In [151]:
normal_dic = {
    "normal" : 0,
    "above normal" : 1,
    "well above normal" : 2,
}

true_false_dic = {
    True : 1,
    False : 0,
}

BMI_dic = {
    "underweight" : 0,
    "normal" : 1,
    "overweight" : 2, 
    "obese": 3, 
    "obese2" : 4, 
    "obese3" : 5
}

blood_pressure_dic = {
    "normal" : 0,
    "elevated" : 1, 
    "highbloodpressure": 2, 
    "highbloodpressure2": 3,
    "hypertensive" : 4
}

df2['cardiovascular'] = df2['cardiovascular'].apply(lambda x : true_false_dic[x])
df2['smoke'] = df2['smoke'].apply(lambda x : true_false_dic[x])
df2['alchohol'] = df2['alchohol'].apply(lambda x : true_false_dic[x])
df2['active'] = df2['active'].apply(lambda x : true_false_dic[x])

df2['cholestrol'] = df2['cholestrol'].apply(lambda x : normal_dic[x])
df2['gloucose'] = df2['gloucose'].apply(lambda x : normal_dic[x])
df2['BMI_class'] = df2['BMI_class'].apply(lambda x : BMI_dic[x])
df2['ap_hi_class'] = df2['ap_hi_class'].apply(lambda x : blood_pressure_dic[x])
df2['ap_lo_class'] = df2['ap_lo_class'].apply(lambda x : blood_pressure_dic[x])
df2

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active,BMI_class,ap_hi_class,ap_lo_class
0,50.4,0,168.0,62.0,0,23.0,110,80,0,0,0,0,1,1,0,4
1,55.4,1,156.0,85.0,1,46.0,140,90,2,0,0,0,1,5,3,3
2,51.7,1,165.0,64.0,1,25.0,130,70,2,0,0,0,0,2,2,1
3,47.9,1,156.0,56.0,0,20.0,100,60,0,0,0,0,0,1,0,1
4,60.0,1,151.0,67.0,0,30.0,120,80,1,1,0,0,0,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,0,168.0,76.0,0,34.0,120,80,0,0,1,0,1,3,1,4
63912,61.9,1,158.0,126.0,1,100.0,140,90,1,1,0,0,1,5,3,3
63913,52.2,0,183.0,105.0,1,60.0,180,90,2,0,0,1,0,5,4,3
63914,61.5,1,163.0,72.0,1,32.0,135,80,0,2,0,0,0,3,2,4


In [103]:
# set X and y values
X = df2.drop(['cardiovascular', 'BMI', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [104]:
# split data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators = 200,
                                          learning_rate = learning_rate,
                                          max_depth = 11,
                                            max_features = 5,
                                          random_state = 9)
    classifier.fit(X_train_scaled, y_train)
    print('Learning Rate: ', learning_rate)
    
    print("Accuracy Score: {0:.3f}".format(classifier.score(X_train_scaled, y_train)))
    print("Validation Score: {0:.3f}".format(classifier.score(X_test_scaled, y_test)))
    print()
    

Learning Rate:  0.05
Accuracy Score: 0.859
Validation Score: 0.720

Learning Rate:  0.1
Accuracy Score: 0.920
Validation Score: 0.711

Learning Rate:  0.25
Accuracy Score: 0.986
Validation Score: 0.690

Learning Rate:  0.5
Accuracy Score: 0.995
Validation Score: 0.679

Learning Rate:  0.75
Accuracy Score: 0.995
Validation Score: 0.674

Learning Rate:  1
Accuracy Score: 0.932
Validation Score: 0.641



In [106]:
# use 134 estimators?
# use 0.025 learning rate
# use 8 max_depth
# use 5 features
classifier = GradientBoostingClassifier(n_estimators = 134,
                                          learning_rate = 0.025,
                                          max_depth = 8,
                                            max_features = 5,
                                          random_state = 9)
classifier.fit(X_train_scaled, y_train)

GradientBoostingClassifier(learning_rate=0.025, max_depth=8, max_features=5,
                           n_estimators=134, random_state=9)

In [107]:
print("Accuracy Score: {0:.3f}".format(classifier.score(X_train_scaled, y_train)))
print("Validation Score: {0:.6f}".format(classifier.score(X_test_scaled, y_test)))

Accuracy Score: 0.755
Validation Score: 0.728519


In [90]:
# bmi 

# set X and y values
X = df2.drop(['cardiovascular', 'height', 'weight', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

classifier = GradientBoostingClassifier(n_estimators = 128,
                                          learning_rate = 0.025,
                                          max_depth = 8,
                                            max_features = 5,
                                          random_state = 9)
classifier.fit(X_train_scaled, y_train)

print("Validation Score: {0:.6f}".format(classifier.score(X_test_scaled, y_test)))

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


Validation Score: 0.728394


In [91]:
#bucket bmi
X = df2.drop(['cardiovascular', 'BMI', 'height', 'weight', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

classifier = GradientBoostingClassifier(n_estimators = 128,
                                          learning_rate = 0.025,
                                          max_depth = 8,
                                            max_features = 5,
                                          random_state = 9)
classifier.fit(X_train_scaled, y_train)

print("Validation Score: {0:.6f}".format(classifier.score(X_test_scaled, y_test)))

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


Validation Score: 0.728206


In [92]:
#bucket blood pressure
X = df2.drop(['cardiovascular', 'BMI', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular']


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

classifier = GradientBoostingClassifier(n_estimators = 128,
                                          learning_rate = 0.025,
                                          max_depth = 8,
                                            max_features = 5,
                                          random_state = 9)
classifier.fit(X_train_scaled, y_train)

print("Validation Score: {0:.6f}".format(classifier.score(X_test_scaled, y_test)))

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


Validation Score: 0.728519


In [111]:
#bucket both
X = df2.drop(['cardiovascular', 'BMI', 'height', 'weight', 'ap_hi', 'ap_lo'], 1).values
y = df2['cardiovascular']


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

classifier = GradientBoostingClassifier(n_estimators = 134,
                                          learning_rate = 0.025,
                                          max_depth = 8,
                                            max_features = 5,
                                          random_state = 9)
classifier.fit(X_train_scaled, y_train)

print("Validation Score: {0:.6f}".format(classifier.score(X_test_scaled, y_test)))

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


Validation Score: 0.729833


In [112]:
importances = classifier.feature_importances_
X = df2.drop(['cardiovascular', 'BMI', 'height', 'weight', 'ap_hi', 'ap_lo'], 1)
sorted(zip(importances, X.columns), reverse = True)

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


[(0.5586109705941251, 'ap_hi_class'),
 (0.2023270752756557, 'age'),
 (0.10150731203940865, 'cholestrol'),
 (0.041197814357148366, 'ap_lo_class'),
 (0.038305990200437234, 'BMI_class'),
 (0.015600641494479765, 'gloucose'),
 (0.015267491126989365, 'active'),
 (0.009923248802610885, 'gender'),
 (0.009896276826199394, 'smoke'),
 (0.0073631792829456436, 'alchohol')]

## XGB Classifier and Naive Bayes - practice code

In [155]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier()
xgb_clf.fit(X_train_scaled, y_train)

print("Validation Score: {0:.6f}".format(xgb_clf.score(X_test_scaled, y_test)))

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:57:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Validation Score: 0.721071


In [225]:
from sklearn.naive_bayes import MultinomialNB

X = df2.drop(['cardiovascular', 'BMI', 'height', 'weight', 'ap_hi', 'ap_lo'], 1).values
y = df2['cardiovascular']


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

gnb = MultinomialNB()
gnb.fit(X_train, y_train)

print("Validation Score: {0:.6f}".format(gnb.score(X_test_scaled, y_test)))

Validation Score: 0.690719


C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [226]:
gnb.classes_

array([0, 1], dtype=int64)

In [190]:
gnb.partial_fit(X_train, y_train)
print("Validation Score: {0:.6f}".format(gnb.score(X_test_scaled, y_test)))

Validation Score: 0.690719


In [152]:
df2['age'] = df2['age'] * 365

In [153]:
df2['age'] = df2['age'].astype(int)

In [182]:
X = df2.drop(['cardiovascular', 'BMI', 'height', 'weight', 'ap_hi', 'ap_lo'], 1).values
y = df2['cardiovascular']


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

gnb = CategoricalNB()
gnb.fit(X_train, y_train)

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


CategoricalNB()

In [183]:
print("Validation Score: {0:.6f}".format(gnb.score(X_test, y_test)))

Validation Score: 0.612429


In [176]:
df2['ap_lo_class'].sort_values()

15769    1
17095    1
17093    1
54421    1
35885    1
        ..
27366    4
27367    4
27368    4
27410    4
63915    4
Name: ap_lo_class, Length: 63916, dtype: int64

In [161]:
df2

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active,BMI_class,ap_hi_class,ap_lo_class
0,18396,0,168.0,62.0,0,23.0,110,80,0,0,0,0,1,1,0,4
1,20221,1,156.0,85.0,1,46.0,140,90,2,0,0,0,1,5,3,3
2,18870,1,165.0,64.0,1,25.0,130,70,2,0,0,0,0,2,2,1
3,17483,1,156.0,56.0,0,20.0,100,60,0,0,0,0,0,1,0,1
4,21900,1,151.0,67.0,0,30.0,120,80,1,1,0,0,0,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,19235,0,168.0,76.0,0,34.0,120,80,0,0,1,0,1,3,1,4
63912,22593,1,158.0,126.0,1,100.0,140,90,1,1,0,0,1,5,3,3
63913,19053,0,183.0,105.0,1,60.0,180,90,2,0,0,1,0,5,4,3
63914,22447,1,163.0,72.0,1,32.0,135,80,0,2,0,0,0,3,2,4
